In [51]:
import bioread
import pandas as pd
acq_file = bioread.read_file(r'C:\Users\USER\Desktop\TIM_115_1.2.24.acq')
channels_to_extract = ['EMG - EMG100C', 'C5 - Expression' ]
data = {}
for channel_name in channels_to_extract:
    channel = acq_file.named_channels[channel_name]
    data[channel_name] = channel.data

df = pd.DataFrame(data)
df.head()

,EMG - EMG100C,C5 - Expression
0,-0.962982,0.0
1,-1.053619,0.0
2,-1.119690,0.0
3,-1.137238,0.0
4,-1.086121,0.0


In [52]:
from scipy.fft import fft
import numpy as np

df_copy = df.copy()

emg_signal = df_copy['EMG - EMG100C'].values
frequencies = np.fft.fftfreq(len(df_copy['EMG - EMG100C']))

df_copy['Frequencies'] = frequencies

frequency_lower_threshold = 40  # Hz
frequency_upper_threshold = 200  # Hz


print(df_copy)

         EMG - EMG100C  C5 - Expression   Frequencies
0            -0.962982              0.0  0.000000e+00
1            -1.053619              0.0  1.265113e-07
2            -1.119690              0.0  2.530225e-07
3            -1.137238              0.0  3.795338e-07
4            -1.086121              0.0  5.060450e-07
...                ...              ...           ...
7904430      -0.004730              0.0 -6.325563e-07
7904431       0.022888              0.0 -5.060450e-07
7904432       0.050659              0.0 -3.795338e-07
7904433       0.075378              0.0 -2.530225e-07
7904434       0.094757              0.0 -1.265113e-07

[7904435 rows x 3 columns]


downsample to 1 of 2

In [53]:
df = df.iloc[::2]
df = df.dropna()
df = df.reset_index(drop=True)

Deleting all the rows with non-zero number in the column 'C5 - Expression' except the last line in each sequence.

In [54]:

non_zero_indices = df.index[df['C5 - Expression'] != 0]
index_rows_to_change = []
for i in range(len(non_zero_indices)-1):
    if non_zero_indices[i+1] == non_zero_indices[i]+1:
        index_rows_to_change.append(non_zero_indices[i])
for index in index_rows_to_change:
    df.loc[index, 'C5 - Expression'] = 0


Number of rows with value not equal to 0 in 'C5 - Expression': 14644
Number of rows with value not equal to 0 in 'C5 - Expression' after changes: 288


In [55]:
from scipy.fft import fft
import numpy as np

np.fft.fft(df['EMG - EMG100C'])

array([-14607.80853271+6.96331881e-13j, -12590.72492838+5.76412539e+03j,
        -9403.10032555+1.02885917e+04j, ...,
        -4406.07975846-1.32322243e+04j,  -9403.10032555-1.02885917e+04j,
       -12590.72492838-5.76412539e+03j])